In [2]:
import sys
import os
import warnings
warnings.filterwarnings('ignore')
import time
from pyspark.sql import SparkSession, DataFrameWriter
import pyspark.sql.functions as F

In [3]:
# Создание спарк сессии
spark = SparkSession.builder.master("local").enableHiveSupport().appName("extract-transform").getOrCreate()
spark

In [9]:
# Читаем данные из паркета
df = spark.read.format("parquet").load('data_in/competition_data_only_target_pqt/')

In [10]:
# Создаем локальное представление датафрейма, как sql таблицы mts
df.createOrReplaceTempView("mts")

In [16]:
target = spark.read.format("parquet").load('data_in/public_train.pqt')
target.createOrReplaceTempView("target")

In [22]:
target.printSchema()

root
 |-- age: double (nullable = true)
 |-- is_male: string (nullable = true)
 |-- user_id: long (nullable = true)
 |-- __index_level_0__: long (nullable = true)



In [23]:
df_sub_1 = spark.sql("select user_id, max(request_cnt) as max_request_cnt, round(avg(request_cnt), 3) as avg_request_cnt, count(request_cnt)as count_request_cnt "
    " from mts group by user_id")
data_learn = target.select("user_id", "is_male", "age").join(df_sub_1, "user_id", 'left')
df_sub_2 = spark.sql("select user_id, max(request_cnt) as max_night_request_cnt, "
    " round(avg(request_cnt), 3) as avg_night_request_cnt, count(request_cnt)as count_night_request_cnt "
    " from mts where part_of_day = 'night' group by user_id")
data_learn = data_learn.join(df_sub_2, "user_id", 'left')
df_sub_3 = spark.sql("select user_id, max(request_cnt) as max_day_request_cnt, "
    " round(avg(request_cnt), 3) as avg_day_request_cnt, count(request_cnt)as count_day_request_cnt "
    " from mts where part_of_day = 'day' group by user_id")
data_learn = data_learn.join(df_sub_3, "user_id", 'left')
df_sub_4 = spark.sql("select user_id, max(request_cnt) as max_morning_request_cnt, "
    " round(avg(request_cnt), 3) as avg_morning_request_cnt, count(request_cnt)as count_morning_request_cnt "
    " from mts where part_of_day = 'morning' group by user_id")
data_learn = data_learn.join(df_sub_4, "user_id", 'left')
df_sub_5 = spark.sql("select user_id, max(request_cnt) as max_evening_request_cnt, "
    " round(avg(request_cnt), 3) as avg_evening_request_cnt, count(request_cnt)as count_evening_request_cnt "
    " from mts where part_of_day = 'evening' group by user_id")
data_learn = data_learn.join(df_sub_5, "user_id", 'left')
df_sub_6 = spark.sql("select user_id, max(sum_date_request_cnt) as max_sum_date_request_cnt, min(sum_date_request_cnt) as min_sum_date_request_cnt, "
    " round(avg(sum_date_request_cnt), 3) as avg_sum_date_request_cnt"
    " from (select user_id, date, sum(request_cnt) as sum_date_request_cnt from mts group by user_id, date) as t1"
    " group by user_id")
data_learn = data_learn.join(df_sub_6, "user_id", 'left')
df_sub_7 = spark.sql("select user_id, max(sum_date_request_cnt) as max_sum_date_day_request_cnt, min(sum_date_request_cnt) as min_sum_date_day_request_cnt, "
    " round(avg(sum_date_request_cnt), 3) as avg_sum_date_day_request_cnt"
    " from (select user_id, date, sum(request_cnt) as sum_date_request_cnt from mts where part_of_day = 'day' group by user_id, date) as t2"
    " group by user_id")
data_learn = data_learn.join(df_sub_7, "user_id", 'left')
df_sub_8 = spark.sql("select user_id, max(sum_date_request_cnt) as max_sum_date_night_request_cnt, min(sum_date_request_cnt) as min_sum_date_night_request_cnt, "
    "round(avg(sum_date_request_cnt), 3) as avg_sum_date_night_request_cnt"
    " from (select user_id, date, sum(request_cnt) as sum_date_request_cnt from mts where part_of_day = 'night' group by user_id, date) as t3"
    " group by user_id")
data_learn = data_learn.join(df_sub_8, "user_id", 'left')
df_sub_9 = spark.sql("select user_id, max(sum_date_request_cnt) as max_sum_date_morning_request_cnt, min(sum_date_request_cnt) as min_sum_date_morning_request_cnt, "
    " round(avg(sum_date_request_cnt), 3) as avg_sum_date_morning_request_cnt"
    " from (select user_id, date, sum(request_cnt) as sum_date_request_cnt from mts where part_of_day = 'morning' group by user_id, date) as t4"
    " group by user_id")
data_learn = data_learn.join(df_sub_9, "user_id", 'left')
df_sub_10 = spark.sql("select user_id, max(sum_date_request_cnt) as max_sum_date_evening_request_cnt, min(sum_date_request_cnt) as min_sum_date_evening_request_cnt, "
    " round(avg(sum_date_request_cnt), 3) as avg_sum_date_evening_request_cnt"
    " from (select user_id, date, sum(request_cnt) as sum_date_request_cnt from mts where part_of_day = 'evening' group by user_id, date) as t5"
    " group by user_id")
data_learn = data_learn.join(df_sub_10, "user_id", 'left')
df_sub_11 = spark.sql("select user_id, count(date) as count_date"
    " from (select user_id, date from mts group by user_id, date) as t6 group by user_id")
data_learn = data_learn.join(df_sub_11, "user_id", 'left')
df_sub_12 = spark.sql("select user_id, count(date) as count_day_date"
    " from (select user_id, date from mts where part_of_day = 'day' group by user_id, date) as t7 group by user_id")
data_learn = data_learn.join(df_sub_12, "user_id", 'left')
df_sub_13 = spark.sql("select user_id, count(date) as count_night_date"
    " from (select user_id, date from mts where part_of_day = 'night' group by user_id, date) as t8 group by user_id")
data_learn = data_learn.join(df_sub_13, "user_id", 'left')
df_sub_14 = spark.sql("select user_id, count(date) as count_morning_date"
    " from (select user_id, date from mts where part_of_day = 'morning' group by user_id, date) as t9 group by user_id")
data_learn = data_learn.join(df_sub_14, "user_id", 'left')
df_sub_15 = spark.sql("select user_id, count(date) as count_evening_date"
    " from (select user_id, date from mts where part_of_day = 'evening' group by user_id, date) as t10 group by user_id")
data_learn = data_learn.join(df_sub_15, "user_id", 'left')
df_sub_16 = spark.sql("select user_id, avg(count_part_of_day_date) as avg_count_part_of_day_date,"
    " max(count_part_of_day_date) as max_count_part_of_day_date, min(count_part_of_day_date) as min_count_part_of_day_date"
    " from (select user_id, date, count(part_of_day) as count_part_of_day_date"
    " from (select user_id, date, part_of_day from mts group by user_id, date, part_of_day) as t11"
    " group by user_id, date) as t12 group by user_id")
data_learn = data_learn.join(df_sub_16, "user_id", 'left')
df_sub_17 = spark.sql("select user_id, avg(lag_date) as avg_lag_date, max(lag_date) as max_lag_date, min(lag_date) as min_lag_date"
    " from (select user_id, int(date - lag(date) over (partition by user_id order by date)) as lag_date"
    " from (select user_id, date from mts group by user_id, date order by user_id, date) as t13) as t14"
    " group by user_id order by user_id")
data_learn = data_learn.join(df_sub_17, "user_id", 'left')
df_sub_18 = spark.sql("select user_id, count(region_name) as count_region_name"
    " from (select user_id, region_name from mts group by user_id, region_name) as t15"
    " group by user_id order by user_id")
data_learn = data_learn.join(df_sub_18, "user_id", 'left')
df_sub_19 = spark.sql("select user_id, count(city_name) as count_city_name"
    " from (select user_id, city_name from mts group by user_id, city_name) as t16"
    " group by user_id order by user_id")
data_learn = data_learn.join(df_sub_19, "user_id", 'left')

In [24]:
%%time
data_learn.write.parquet(path="data_out/data_for_learn_parquet_last", mode="overwrite")

CPU times: user 93.9 ms, sys: 44.1 ms, total: 138 ms
Wall time: 11min 8s


In [25]:
data_test = spark.read.format("parquet").load('data_out/data_for_learn_parquet_last')

In [26]:
data_test.show(3, vertical=True)

-RECORD 0----------------------------------------------
 user_id                          | 0                  
 is_male                          | 0                  
 age                              | 35.0               
 max_request_cnt                  | 5                  
 avg_request_cnt                  | 1.473              
 count_request_cnt                | 131                
 max_night_request_cnt            | 1                  
 avg_night_request_cnt            | 1.0                
 count_night_request_cnt          | 1                  
 max_day_request_cnt              | 5                  
 avg_day_request_cnt              | 1.911              
 count_day_request_cnt            | 56                 
 max_morning_request_cnt          | 2                  
 avg_morning_request_cnt          | 1.15               
 count_morning_request_cnt        | 20                 
 max_evening_request_cnt          | 3                  
 avg_evening_request_cnt          | 1.148       

In [15]:
# data_learn = spark.sql("with sub_1 as"
#     " (select user_id, max(request_cnt) as max_request_cnt, round(avg(request_cnt), 3) as avg_request_cnt, count(request_cnt)as count_request_cnt "
#     " from mts group by user_id)"
#     ", sub_2 as (select user_id, max(request_cnt) as max_night_request_cnt, "
#     " round(avg(request_cnt), 3) as avg_night_request_cnt, count(request_cnt)as count_night_request_cnt "
#     " from mts where part_of_day = 'night' group by user_id)"
    # ", sub_3 as (select user_id, max(request_cnt) as max_day_request_cnt, "
    # " round(avg(request_cnt), 3) as avg_day_request_cnt, count(request_cnt)as count_day_request_cnt "
    # " from mts where part_of_day = 'day' group by user_id)"
    # ", sub_4 as (select user_id, max(request_cnt) as max_morning_request_cnt, "
    # " round(avg(request_cnt), 3) as avg_morning_request_cnt, count(request_cnt)as count_morning_request_cnt "
    # " from mts where part_of_day = 'morning' group by user_id)"
    # ", sub_5 as (select user_id, max(request_cnt) as max_evening_request_cnt, "
    # " round(avg(request_cnt), 3) as avg_evening_request_cnt, count(request_cnt)as count_evening_request_cnt "
    # " from mts where part_of_day = 'evening' group by user_id)"
    # ", sub_6 as (select user_id, max(sum_date_request_cnt) as max_sum_date_request_cnt, min(sum_date_request_cnt) as min_sum_date_request_cnt, "
    # " round(avg(sum_date_request_cnt), 3) as avg_sum_date_request_cnt"
    # " from (select user_id, date, sum(request_cnt) as sum_date_request_cnt from mts group by user_id, date) as t1"
    # " group by user_id)"
    # ", sub_7 as (select user_id, max(sum_date_request_cnt) as max_sum_date_day_request_cnt, min(sum_date_request_cnt) as min_sum_date_day_request_cnt, "
    # " round(avg(sum_date_request_cnt), 3) as avg_sum_date_day_request_cnt"
    # " from (select user_id, date, sum(request_cnt) as sum_date_request_cnt from mts where part_of_day = 'day' group by user_id, date) as t2"
    # " group by user_id)"
    # ", sub_8 as (select user_id, max(sum_date_request_cnt) as max_sum_date_night_request_cnt, min(sum_date_request_cnt) as min_sum_date_night_request_cnt, "
    # "round(avg(sum_date_request_cnt), 3) as avg_sum_date_night_request_cnt"
    # " from (select user_id, date, sum(request_cnt) as sum_date_request_cnt from mts where part_of_day = 'night' group by user_id, date) as t3"
    # " group by user_id)"
    # ", sub_9 as (select user_id, max(sum_date_request_cnt) as max_sum_date_morning_request_cnt, min(sum_date_request_cnt) as min_sum_date_morning_request_cnt, "
    # " round(avg(sum_date_request_cnt), 3) as avg_sum_date_morning_request_cnt"
    # " from (select user_id, date, sum(request_cnt) as sum_date_request_cnt from mts where part_of_day = 'morning' group by user_id, date) as t4"
    # " group by user_id)"
    # ", sub_10 as (select user_id, max(sum_date_request_cnt) as max_sum_date_evening_request_cnt, min(sum_date_request_cnt) as min_sum_date_evening_request_cnt, "
    # " round(avg(sum_date_request_cnt), 3) as avg_sum_date_evening_request_cnt"
    # " from (select user_id, date, sum(request_cnt) as sum_date_request_cnt from mts where part_of_day = 'evening' group by user_id, date) as t5"
    # " group by user_id)"
    # ", sub_11 as (select user_id, count(date) as count_date"
    # " from (select user_id, date from mts group by user_id, date) as t6 group by user_id)"
    # ", sub_12 as (select user_id, count(date) as count_day_date"
    # " from (select user_id, date from mts where part_of_day = 'day' group by user_id, date) as t7 group by user_id)"
    # ", sub_13 as (select user_id, count(date) as count_night_date"
    # " from (select user_id, date from mts where part_of_day = 'night' group by user_id, date) as t8 group by user_id)"
    # ", sub_14 as (select user_id, count(date) as count_morning_date"
    # " from (select user_id, date from mts where part_of_day = 'morning' group by user_id, date) as t9 group by user_id)"
    # ", sub_15 as (select user_id, count(date) as count_evening_date"
    # " from (select user_id, date from mts where part_of_day = 'evening' group by user_id, date) as t10 group by user_id)"
    # ", sub_16 as (select user_id, avg(count_part_of_day_date) as avg_count_part_of_day_date,"
    # " max(count_part_of_day_date) as max_count_part_of_day_date, min(count_part_of_day_date) as min_count_part_of_day_date"
    # " from (select user_id, date, count(part_of_day) as count_part_of_day_date"
    # " from (select user_id, date, part_of_day from mts group by user_id, date, part_of_day) as t11"
    # " group by user_id, date) as t12 group by user_id)"
    # ", sub_17 as (select user_id, avg(lag_date) as avg_lag_date, max(lag_date) as max_lag_date, min(lag_date) as min_lag_date"
    # " from (select user_id, int(date - lag(date) over (partition by user_id order by date)) as lag_date"
    # " from (select user_id, date from mts group by user_id, date order by user_id, date) as t13) as t14"
    # " group by user_id order by user_id)"
    # ", sub_18 as (select user_id, count(region_name) as count_region_name"
    # " from (select user_id, region_name from mts group by user_id, region_name) as t15"
    # " group by user_id order by user_id)"
    # ", sub_19 as (select user_id, count(city_name) as count_city_name"
    # " from (select user_id, city_name from mts group by user_id, city_name) as t16"
    # " group by user_id order by user_id)"                     
    # ).show(3, vertical=True)

-RECORD 0-------------------
 user_id           | 99002  
 age               | 41.0   
 is_male           | 0      
 max_request_cnt   | 8      
 avg_request_cnt   | 1.34   
 count_request_cnt | 639    
-RECORD 1-------------------
 user_id           | 155506 
 age               | 33.0   
 is_male           | 0      
 max_request_cnt   | 5      
 avg_request_cnt   | 1.727  
 count_request_cnt | 22     
-RECORD 2-------------------
 user_id           | 188276 
 age               | 35.0   
 is_male           | 1      
 max_request_cnt   | 4      
 avg_request_cnt   | 1.414  
 count_request_cnt | 111    
only showing top 3 rows



In [11]:
data_test.show(3, vertical=True)

-RECORD 0------------------
 user_id           | 0     
 age               | 35.0  
 is_male           | 0     
 max_request_cnt   | 5     
 avg_request_cnt   | 1.473 
 count_request_cnt | 131   
-RECORD 1------------------
 user_id           | 1     
 age               | 41.0  
 is_male           | 0     
 max_request_cnt   | 6     
 avg_request_cnt   | 1.496 
 count_request_cnt | 700   
-RECORD 2------------------
 user_id           | 2     
 age               | 25.0  
 is_male           | 0     
 max_request_cnt   | 4     
 avg_request_cnt   | 1.154 
 count_request_cnt | 356   
only showing top 3 rows

